In [ ]:
'''

#Mount drive


class FileSystem:
  def __init__(self, colab_dir="PersonTracking", local_dir="./", data_dir="data"): # replace with dlav path
    IN_COLAB = 'google.colab' in sys.modules
    if (IN_COLAB):
      from google.colab import drive
      drive.mount('/gdrive')
      self.root_dir = os.path.join("/gdrive/My Drive/", colab_dir)
    else:
      self.root_dir = local_dir
    self.data_dir = data_dir
    self.change_directory = False

  def data_path(self, name):
    return os.path.join(self.data_dir, name) if self.change_directory else os.path.join(self.root_dir, self.data_dir, name)

  def path(self, name):
    return os.path.join("./", name) if self.change_directory else os.path.join("./", self.root_dir, name)

  def cd(self):
    %cd {self.root_dir}
    %ls
    self.change_directory = True

fs = FileSystem()
fs.cd()
'''

In [ ]:
import time
import datetime
import argparse
import random
import os, sys


import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import numpy as np

from tqdm import tqdm 

import io
import PIL
from PIL import Image
import requests
import cv2
import shutil

#from google.colab.patches import cv2_imshow
import os.path
import glob
import warnings


In [ ]:
!pwd

In [4]:
try:
  shutil.rmtree('reid-strong-baseline')
except OSError as e: 
  pass

try:
  shutil.rmtree('YOLOv3-pedestrian')
except OSError as e: 
  pass

#!git clone https://github.com/nodiz/reid-strong-baseline.git
!git clone https://github.com/nodiz/YOLOv3-pedestrian.git
#!cd YOLOv3-pedestrian && git checkout AnchorsForVideo

!cp /home/fabien/Desktop/DLfAD/YOLOv3-pedestrian/weights/yolov3_ckpt_current_50.pth YOLOv3-pedestrian/weights

Receiving objects:  23% (158/673), 7.29 MiB | 883.00 KiB/sReceiving objects:  38% (256/673), 10.90 MiB | 864.00 KiB/sremote: Total 673 (delta 69), reused 67 (delta 24), pack-reused 552
Receiving objects: 100% (673/673), 13.29 MiB | 763.00 KiB/s, done.
Resolving deltas: 100% (409/409), done.
cp: /home/fabien/Desktop/DLfAD/YOLOv3-pedestrian/weights/yolov3_ckpt_current_50.pth: No such file or directory


**Affiliation**

In [5]:
def crop_affilition(distmat,dist_thld,framenr,gallery_path,name_query,name_gallery):

    #For distmat: rows: Querys, Collums: Gallery
    
    maxiter=np.shape(distmat)[0]
    gallength=len(os.listdir(gallery_path))
    querynr=0
    
    query = np.arange(0,maxiter)
    aff_mat=np.zeros((maxiter,2))

    while querynr < maxiter:
        
          q_min=np.amin(distmat)
          result = np.where(distmat == q_min)

          result=(result[0][0], result[1][0])
          idx=(result[0], result[1])
          #print(distmat[idx], dist_thld)
        

          if (distmat[idx]<dist_thld and querynr<gallength):

            #print('Put Query '+str(result[0])+' into Gallery ' +f'{result[1]}')

            shutil.move(name_query[result[0]], name_gallery[result[1]][:-9]+'/'+str(framenr)+'.jpg')
            aff_mat[querynr,:]=[result[0],int(name_gallery[result[1]][-13:-9])]

            for i in range(0,np.shape(name_gallery)[0]):
                if name_gallery[i][-13:-9] == name_gallery[result[1]][-13:-9]:
                    distmat[:,i]=float("inf")



            distmat[:,result[1]]=float("inf")
            distmat[result[0],:]=float("inf")
            query = np.delete(query, np.where(query == result[0]))
            querynr+=1

          else:
            #put remaining queries in new folder

            for i in query:

                #print('Put Query '+str(i)+' into new Gallery ' +f'{gallength}')
                os.makedirs(name_gallery[0][:-14]+'/'+f'{gallength:04}', exist_ok = True)      
                shutil.move(name_query[i], name_gallery[0][:-14]+'/'+f'{gallength:04}'+'/'+str(framenr)+'.jpg')
                aff_mat[querynr,:]=[i,gallength]
                gallength+=1
                querynr+=1

    return aff_mat             


**RE-ID Function**

In [ ]:
# Build model
#!ls #/gdrive#/.shortcut-targets-by-id/ #1xcANAd3HJkfxNCBAtjJOyodOgmX0iO8b/PersonTracking
from reidLib.modeling.baseline import Baseline

__file__ = 'reidLib/modeling'
sys.path.append(os.path.dirname(__file__))

def build_model(num_classes):
    #     model = Baseline(num_classes, cfg.MODEL.LAST_STRIDE, cfg.MODEL.PRETRAIN_PATH, cfg.MODEL.NECK, cfg.TEST.NECK_FEAT \ after)
    model = Baseline(num_classes, 1, "WeightsReid/resnet50-19c8e357.pth", 'bnneck', 'after', 'resnet50', 'imageNet')   # maybe try self instead of imageNet
    return model

device = 'cuda'
model = build_model(1041)  # 1041 identities in training dataset
model.eval()  # evaluation mode
#model=torch.load('WeightsReid/resnet50_ibn_a_center_param_120.pth')
model.load_param('WeightsReid/WeightFab/center/resnet50_model_100.pth')
model.to(device);

In [ ]:
# Prepare transform

normalize_transform = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = T.Compose([
    T.Resize([256, 128]),
    T.ToTensor(),
    normalize_transform
])


#Dataset  

class ImageDataset(Dataset):
    """Image Person ReID Dataset"""

    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path = self.dataset[index]
        img = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img

# Dataloader

def val_collate_fn(batch):
    return torch.stack(batch, dim=0)


In [ ]:
def reid(gallery_path='pic/*', query_path='pic/*'):
    
    #Load images query 
    Gallery_size = 140
    imgs_query = sorted(glob.glob(os.path.join(query_path, "*.*"))) 
    num_query = np.shape(imgs_query)[0]  # first x images in dataset

    # Load images gallery
    imgs_gallery = sorted(glob.iglob(os.path.join(gallery_path, "*.*")),key=os.path.getctime, reverse=True) #pic = gallery_path

    if np.shape(imgs_gallery)[0]>Gallery_size :
        imgs_gallery = imgs_gallery[:Gallery_size ]
        
    imgs=imgs_query + imgs_gallery

    #print([os.path.basename(x) for x in imgs[:num_query]])
    #print([os.path.basename(x) for x in imgs[num_query:]])

    # build dataset and dataloader  !!!! np.shape(imgs)[0]
    imdatas = ImageDataset(imgs, transform)
    demo_loader = DataLoader(
        imdatas, batch_size=np.shape(imgs)[0], shuffle=False, num_workers=4,
        collate_fn=val_collate_fn
    )


    # model evaluation

    with torch.no_grad():
      for batch in demo_loader:
        batch = batch.to(device)
        feat = model(batch) # (bs, 2048)
        feat_norm = 1
        if feat_norm:
          feat = torch.nn.functional.normalize(feat, dim=1, p=2)
        # query
        qf = feat[:num_query]

        # gallery
        gf = feat[num_query:]

        m, n = qf.shape[0], gf.shape[0]

        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()

        distmat.addmm_(1, -2, qf, gf.t())
        distmat = distmat.cpu().numpy()

        return distmat, imgs_query, imgs_gallery

In [ ]:
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

def show(final):
    print('display')
    cv2.imshow('bla', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

**Bounding Box**

In [ ]:
def bboxdrawer(img_path,txtfile_path,aff_mat, gallery_path, idx):
  
    # Creates bounding box by overwriting raw frame at img_path
    aff_mat=aff_mat[aff_mat[:,0].argsort()]

    img = np.array(Image.open(img_path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)
  
    cmap = plt.get_cmap("tab20b")

    hsvBig = plt.get_cmap('hsv', 512)
    
    cmap = ListedColormap(hsvBig(np.linspace(0, 1, 256)))
  
    colors = [cmap(i) for i in np.linspace(0, 1, 256)]
    
    bboxmatrix = np.loadtxt(txtfile_path)
    
    if np.shape(aff_mat)[0] == 1:
        bboxmatrix[0] = aff_mat[0][1]
        bboxmatrix=[bboxmatrix]
        bb_boxfor=bboxmatrix
    else:    
        bboxmatrix[:,0]=aff_mat[:,1]
        bboxmatrix=[bboxmatrix]

        bb_boxfor=bboxmatrix[0]    

    bbox_colors = np.array(colors)[idx]    
    
    for label,x1,x2,y1,y2 in bb_boxfor:

      ''' 
      label=bboxmatrix[j,0]
      x1=bboxmatrix[j,1]
      x2=bboxmatrix[j,2]
      y1=bboxmatrix[j,3]
      y2=bboxmatrix[j,4]
      '''
      
      box_w = x2 - x1
      box_h = y2 - y1
      color = bbox_colors[int(label-1)]
      #print(color)
      bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=1, edgecolor=color, facecolor="none")
      ax.add_patch(bbox)
      plt.text(
                x1,
                y1,
                s=int(label),
                color="white",
                verticalalignment="top",
                bbox={"color": color, "pad": 0},
            )
    plt.axis("off")
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig(img_path, bbox_inches="tight", pad_inches=0.0)
    #plt.close()

#bboxdrawer('Frames/MOT16-11-raw/frame2.jpg','DetectedImages/MOT16-11-raw/Detection2.txt')
#bboxdrawer('Frames/MOT16-11-raw/frame3.jpg','DetectedImages/MOT16-11-raw/Detection3.txt')



**Detection Algorithm**

In [ ]:
#clear sample folder in YoloV3


files = glob.glob('YOLOv3-pedestrian/data/samples/*')
for f in files:
    os.remove(f)
files = glob.glob('YOLOv3-pedestrian/output/*')
for f in files:
    os.remove(f)



#define and read video

videoname = 'MOT16-11-raw'
threshold=0.13 #0.13


# Color BBOX
    
n_cls = 150 #Maximum color 
random.seed(1)
index=np.linspace(0,255, n_cls).astype(int)
np.random.shuffle(index)

# data aug
hsv=True
wbalance=True

#delete at every execution folder such that code can be executed again
try:
  shutil.rmtree('DetectedImages/'+videoname)
except OSError as e: 
  pass

frame_path='Frames/'+videoname
try:
  shutil.rmtree(frame_path)
except OSError as e: 
  pass
os.makedirs(frame_path, exist_ok=True)

videoframe = cv2.VideoCapture('VideoToTrack/'+videoname+'.webm')
framenr=0
firstdetection=True

if (videoframe.isOpened()== False): 
  print("Error opening video stream or file")
while(videoframe.isOpened()):
  # Capture frame-by-frame
  ret, frame = videoframe.read()
  if ret == True:
    framenr+=1
    
    # Display the resulting frame
    

    if wbalance ==True:
        frame = white_balance(frame)

    if hsv==True:

        hsvImg = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        #multiple by a factor to change the saturation
        hsvImg[...,1] = hsvImg[...,1]*1.5

        #multiple by a factor of less than 1 to reduce the brightness 
        hsvImg[...,2] = hsvImg[...,2]*0.99

        frame=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2BGR)
        
    #cv2_imshow(frame)
    imagename='YOLOv3-pedestrian/data/samples/'+videoname+str(framenr)+'.jpg'
    cv2.imwrite(imagename, frame)
    cv2.imwrite(frame_path+'/frame'+str(framenr)+'.jpg', frame)

    
    !python3 YOLOv3-pedestrian/detectbbox.py --conf_thres 0.9 --img_size 540 --image_folder YOLOv3-pedestrian/data/samples --model_def YOLOv3-pedestrian/config/yolov3-custom.cfg --output_dir YOLOv3-pedestrian/output --weights_path YOLOv3-pedestrian/weights/yolov3_ckpt_current_50.pth --class_path YOLOv3-pedestrian/data/classes.names
    os.remove('YOLOv3-pedestrian/data/samples/'+videoname+str(framenr)+'.jpg')
    
    os.makedirs('DetectedImages/'+videoname, exist_ok=True)
    os.makedirs('DetectedImages/'+videoname+'-txtfiles', exist_ok=True)
    

    #if detection is made do reidentification else put a warning that nothing was
    if os.path.exists('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt') and np.size(np.loadtxt('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt')) != 0:
        print(firstdetection)
        shutil.move('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt', 'DetectedImages/'+videoname+'-txtfiles/Detection'+str(framenr)+'.txt')

          #for the first frame put every detected person crop in separate folder
        if firstdetection==True: 

            DIR = 'YOLOv3-pedestrian/output/'
            detection = sorted(glob.glob(os.path.join(DIR, "*.jpg")))
            print(detection, range(1,np.size(detection)+1) )
            for pid1 in range(0,np.size(detection)):
              os.mkdir('DetectedImages/'+videoname+'/'+f'{pid1:04}')
              shutil.move('YOLOv3-pedestrian/output/'+f'{pid1+1:04}'+'.jpg', 'DetectedImages/'+videoname+'/'+f'{pid1:04}'+'/'+f'{framenr:04}'+'.jpg' )
            
            firstdetection=False

        else: # np.size(np.loadtxt('DetectedImages/'+videoname+'-txtfiles/Detection'+str(framenr)+'.txt')) != 0:
          #implement person reid here and affilate image to correct folder

            distmat, name_query, name_gallery = reid(gallery_path = 'DetectedImages/'+videoname+'/*', query_path='YOLOv3-pedestrian/output/')
            print('QUER', np.shape(name_query))
            print('GAL', np.shape(name_gallery))
            amat=crop_affilition(distmat= distmat, dist_thld = threshold, framenr=f'{framenr:04}', gallery_path = 'DetectedImages/'+videoname, name_query = name_query, name_gallery = name_gallery) 
            bboxdrawer(frame_path+'/frame'+str(framenr)+'.jpg','DetectedImages/'+videoname+'-txtfiles/'+'Detection'+str(framenr)+'.txt',amat,gallery_path = 'DetectedImages/'+videoname, idx=index)      
            
            
            
            torch.cuda.empty_cache()
    elif os.path.exists('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt') and np.size(np.loadtxt('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt')) == 0: #pippo
        os.remove('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt')
        warnings.warn('no detection in frame number '+str(framenr))
    else:
        warnings.warn('no detection in frame number '+str(framenr))
    
  # Break the loop when video is finished
  else: #here
    break

# When everything done, release the video capture object

videoframe.release()
# Closes all the frames
cv2.destroyAllWindows()


In [ ]:
np.shape('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.txt') != ()

In [ ]:
!pwd
!ls DetectedImages/MOT16-10-raw/

In [ ]:
   '''
    
    aff_mat=amat
    
    aff_mat=aff_mat[aff_mat[:,0].argsort()]

    cmap = plt.get_cmap("tab20b")

    hsvBig = plt.get_cmap('hsv', 512)
    
    cmap = ListedColormap(hsvBig(np.linspace(0, 1, 256)))
  
    colors = [cmap(i) for i in np.linspace(0, 1, 256)]
    
    bboxmatrix = np.loadtxt('DetectedImages/'+videoname+'-txtfiles/'+'Detection'+str(framenr)+'.txt')
    
    if np.shape(aff_mat)[0] == 1:
        bboxmatrix[0] = aff_mat[0][1]
        bboxmatrix=[bboxmatrix]
        bb_boxfor=bboxmatrix
    else:    
        bboxmatrix[:,0]=aff_mat[:,1]
        bb_boxfor=bboxmatrix[0]
   
    #if type(bboxmatrix)==np.ndarray:
      #print('pippo')
      #bboxmatrix=[bboxmatrix]


    n_cls=len(os.listdir('DetectedImages/'+videoname))
    
    
    random.seed(1)
    bbox_colors = random.sample(colors, n_cls)
    
    print(bb_boxfor)
    
    for label,x1,x2,y1,y2 in bb_boxfor:
        print('pippo')
        
    '''

Frame to video

In [ ]:
image_folder = 'Frames/'+videoname +'/'
video_name = 'Video/'+videoname+'_BoundingBoxv3.avi'

#images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
#print(images[0:10])
images = sorted(glob.iglob(os.path.join(image_folder, "*.jpg")),key=os.path.getctime, reverse=True)[::-1]
images= images[1:]

frame = cv2.imread(images[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

print(height,width)


for image in images:
    a = cv2.imread(image)
    a = cv2.resize(a, (width,height), interpolation = cv2.INTER_AREA)
    print(image, a.shape)
    video.write(a)

cv2.destroyAllWindows()
video.release()


Tests (not important)

In [ ]:
''''

videoname = 'MOT16-10-raw'
videoframe = cv2.VideoCapture('VideoToTrack/'+videoname+'.webm')

framenr=0
while(videoframe.isOpened()):
  ret, frame = videoframe.read()
  if ret ==True:
  #cv2_imshow(frame)
    framenr+=1
    imagename='YOLOv3-pedestrian/data/samples/'+videoname+str(framenr)+'.jpg'
    cv2.imwrite(imagename, frame)

    %cd YOLOv3-pedestrian/  
    !python detect.py --model_def config/yolov3-custom.cfg --weights_path weights/yolov3_ckpt_current_50.pth
    os.remove('data/samples/'+videoname+str(framenr)+'.jpg')
    %cd ..
    os.makedirs('Detection/'+videoname, exist_ok=True)
    shutil.move('YOLOv3-pedestrian/output/'+videoname+str(framenr)+'.png', 'Detection/'+videoname+str(framenr)+'.png' )
    #cv2_imshow( 'Detection/'+videoname+str(framenr)+'.png')
  
  else:
    break
    

videoframe.release()
# Closes all the frames
cv2.destroyAllWindows()
'''

In [ ]:
videoname = 'MOT16-11-raw'
videoframe = cv2.VideoCapture('VideoToTrack/'+videoname+'.webm')


if (videoframe.isOpened()== False): 
  print("Error opening video stream or file")
else:
  ret, frame = videoframe.read()
  cv2.imshow('blabla.jpg',frame)



hsvImg = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

#multiple by a factor to change the saturation
hsvImg[...,1] = hsvImg[...,1]*1.5

#multiple by a factor of less than 1 to reduce the brightness 
hsvImg[...,2] = hsvImg[...,2]*0.5

image=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2BGR)

cv2.imshow('blabla2.jpg',image)

In [ ]:
!pwd
!ls
#os.remove('MOT16-10-raw1.jpg')
!pwd

In [ ]:
!pwd
!ls
#os.remove('MOT16-10-raw1.jpg')
!pwd

In [ ]:
videoname = 'MOT16-11-raw'
videoframe = cv2.VideoCapture('VideoToTrack/'+videoname+'.webm')


if (videoframe.isOpened()== False): 
  print("Error opening video stream or file")
else:
  ret, frame = videoframe.read()
  cv2.imshow('blabla.jpg',frame)



hsvImg = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

#multiple by a factor to change the saturation
hsvImg[...,1] = hsvImg[...,1]*1.5

#multiple by a factor of less than 1 to reduce the brightness 
hsvImg[...,2] = hsvImg[...,2]*0.5

image=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2BGR)

cv2.imshow('blabla2.jpg',image)